In [57]:
import pandas as pd
import numpy as np
import os
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [7]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [8]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [11]:
downloaded = drive.CreateFile({'id': '16qBB5M1R81wW_bEtCs4eJg1SVHsQn9q4'})
downloaded.GetContentFile('cnndh.zip')

In [ ]:
!unzip cnndh.zip

In [ ]:
base_model= MobileNet(weights='imagenet',include_top=False)

In [ ]:
base_model.summary()

In [72]:
CLASSES = 3
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = tf.keras.layers.Dropout(0.4)(x)
predictions =tf.keras.layers.Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [73]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [74]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'Dog_Human/train'
TEST_DIR=r'Dog_Human/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
print(train_generator.class_indices)
print(validation_generator.class_indices)

Found 19810 images belonging to 3 classes.
Found 3504 images belonging to 3 classes.
{'dog': 0, 'human': 1, 'nothd': 2}
{'dog': 0, 'human': 1, 'nothd': 2}


In [75]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [83]:

BATCH_SIZE = 32
model.fit(
    train_generator,
    epochs=2,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
    )

Epoch 1/2
620/620 [==============================] - 501s 809ms/step - loss: 0.0339 - accuracy: 0.9882 - val_loss: 0.0405 - val_accuracy: 0.9857
Epoch 2/2
620/620 [==============================] - 501s 808ms/step - loss: 0.0274 - accuracy: 0.9903 - val_loss: 0.0394 - val_accuracy: 0.9880


In [79]:
from tensorflow.python.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json

In [36]:
img = image.load_img(r'Dog_Human/test/dog/1003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = pred.argmax(axis=-1)
y_classes

[9.9997902e-01 2.1624223e-07 2.0795040e-05]


0

In [ ]:
img = image.load_img(r'Dog_Human/test/human/Sharon_Osbourne_0003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = np.argmax(pred)
y_classes

[0.01346153 0.97722685 0.00931159]


1

In [82]:

from tensorflow.keras.models import model_from_json
model_json = model.to_json()


with open("cnn.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("cnn.h5")

In [44]:
train_generator.class_indices

{'dog': 0, 'human': 1, 'nothd': 2}